In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor


In [10]:
df = pd.read_csv('cleaned_data.csv')


In [11]:
df.head()


,Country,Consultant,Opportunity ID,Account,ISBN 13,Author,Title,Probability,% de Avance,Term,Seats,Units,Revenue,Technology,Course Name,Edition,Course ID,Customer,Client,Coded Probability
0,Mexico,14,5255970,colegio de bachilleres de chihuahua,9.786075e+12,martinez,literatura ii,No_Aceptada,NaN,2021-08-01,10000,8929,1308277.08,Copia física,literatura cobach,1.0,1847125,Potential Customer,3705,0.00
1,Mexico,10,5172372,u-erre universidad regiomontana,9.781338e+12,serway/vuille,intl iac wa fundamentos de fis ica,Closed Sale,1.00,2021-01-01,24,22,1161600.00,WebAssign,fisica 1 prepa u-erre,11.0,1905363,Potential Customer,3681,100.00
2,Mexico,6,4496314,itesm campus monterrey,9.786075e+12,cengage,biblioteca digital magellan,No_Aceptada,NaN,2020-09-01,1000,2,779999.95,Copia física,biblioteca digital,1.0,1873177,Potential Customer,5929,0.00
3,Mexico,0,5745690,unitec campus cuitlahuac,9.786076e+12,galindo,modelos de organización,Evaluation Visit,0.75,2022-05-01,3000,2000,600000.00,Copia física,estructuras organizacionales y gestión del con...,1.0,1934416,Potential Customer,2538,0.75
4,Mexico,0,5745686,unitec campus cuitlahuac,9.786076e+12,galindo,modelos de organización,Evaluation Visit,0.75,2022-01-01,3000,2000,600000.00,Copia física,estructuras organizacionales y gestión del con...,1.0,1934416,Potential Customer,2538,0.75


## Selección de variables

In [12]:
y = df['Revenue']
X = df.drop(['Revenue'], axis=1)


In [13]:
X_train, y_train, X_test, y_test = train_test_split(X, y, random_state=0)


In [14]:
my_tree = DecisionTreeRegressor(max_depth=3, random_state=0)
my_tree.fit(X_train, y_train)


ValueError: could not convert string to float: 'Mexico'